In [1]:
import sys
import cv2
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
from google.colab.patches import cv2_imshow

In [3]:
# Load our images
image1 = cv2.imread("image1.jpg")
image2 = cv2.imread("image2.jpg")
image3 = cv2.imread("image3.jpg")
image4 = cv2.imread("image4.jpg")

images = [image1, image2, image3, image4]

result = images[0]

# Convert images to grayscale
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]
for i in range(len(gray_images)):
    cv2_imshow(gray_images[i])

Output hidden; open in https://colab.research.google.com to view.

In [4]:
#Using ORB detector
orb = cv2.ORB_create()

# Find keypoints and descriptors for each image
keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
keypoints3, descriptors3 = orb.detectAndCompute(image3, None)
keypoints4, descriptors4 = orb.detectAndCompute(image4, None)

# Draw keypoints on images
image1_with_keypoints = cv2.drawKeypoints(image1, keypoints1, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
image2_with_keypoints = cv2.drawKeypoints(image2, keypoints2, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
image3_with_keypoints = cv2.drawKeypoints(image3, keypoints3, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
image4_with_keypoints = cv2.drawKeypoints(image4, keypoints4, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

# Display images with keypoints
cv2_imshow(image1_with_keypoints)
cv2_imshow(image2_with_keypoints)
cv2_imshow(image3_with_keypoints)
cv2_imshow(image4_with_keypoints)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
# Create a BFMatcher (BruteForce) object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors using knn
matches12 = bf.match(descriptors1, descriptors2)
matches23 = bf.match(descriptors2, descriptors3)
matches34 = bf.match(descriptors3, descriptors4)

# Sort the matches based on distance
matches12 = sorted(matches12, key=lambda x: x.distance)
matches23 = sorted(matches23, key=lambda x: x.distance)
matches34 = sorted(matches34, key=lambda x: x.distance)

# Take top 50 matches
good_matches12 = matches12[:50]
good_matches23 = matches23[:50]
good_matches34 = matches34[:50]

In [6]:
# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches12]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches12]).reshape(-1, 1, 2)
H12, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

src_pts = np.float32([keypoints2[m.queryIdx].pt for m in good_matches23]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints3[m.trainIdx].pt for m in good_matches23]).reshape(-1, 1, 2)
H23, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

src_pts = np.float32([keypoints3[m.queryIdx].pt for m in good_matches34]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints4[m.trainIdx].pt for m in good_matches34]).reshape(-1, 1, 2)
H34, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

In [ ]:
# Warp images
panorama = cv2.warpPerspective(image1, H12, (image1.shape[1] + image2.shape[1], image1.shape[0]))
panorama[0:image2.shape[0], 0:image2.shape[1]] = image2

temp = cv2.warpPerspective(image3, H23, (image3.shape[1] + image4.shape[1], image3.shape[0]))
temp[0:image4.shape[0], 0:image4.shape[1]] = image4

final_panorama = cv2.warpPerspective(panorama, H34, (panorama.shape[1] + temp.shape[1], panorama.shape[0]))
final_panorama[0:temp.shape[0], 0:temp.shape[1]] = temp

# Blend images using alpha blending
alpha = 0.5  # You can adjust the alpha value for different blending effects

# Blend image1 and image2
blended_1_2 = cv2.addWeighted(image1, 1 - alpha, image2, alpha, 0)

# Blend blended_1_2 and image3
blended_1_2_3 = cv2.addWeighted(blended_1_2, 1 - alpha, image3, alpha, 0)

# Blend blended_1_2_3 and image4
final_panorama = cv2.addWeighted(blended_1_2_3, 1 - alpha, image4, alpha, 0)

# Display the stitched image with alpha blending
cv2_imshow(final_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()
